Vamos a importar la bases de datos y a convertirlas en objetos de series de Tiempo. $\{X_t\}$


In [ ]:
# get working directory
import os
os.getcwd()

In [ ]:
# librerias
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import sklearn
import openpyxl
from skforecast.ForecasterAutoreg import ForecasterAutoreg
import warnings
print(f"Matplotlib Version: {plt.__version__}")
print(f"Pandas Version: {pd.__version__}")
print(f"Numpy Version: {np.__version__}")
print(f"Sklearn: {sklearn.__version__}")

In [ ]:
# Lectura de la serie
data = pd.read_excel("datos/Exportaciones.xlsx",
                   header = 0, usecols = ['Mes','Total']).iloc[96:].reset_index(drop = True).round()
data['Total'] = data['Total'].astype(int) 
data

In [ ]:
# tipo de datos
print(data.info())

In [ ]:
#mirando los datos
#objeto ts
exportaciones = data['Total']
print(type(exportaciones))
plt.plot(exportaciones)

In [ ]:
print(f'Numero de filas con valores faltantes: {data.isnull().any(axis=1).mean()}')

In [ ]:
data.shape

## PACF

usaremos la funcion de autocorrealcion parcial para darnos una idea de cuantos rezagos usaremos en el modelo


In [ ]:
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(exportaciones,lags=140,method='ywm',alpha=0.01)
pyplot.show()

In [ ]:
import statsmodels.api as sm
pacf =  sm.tsa.stattools.pacf(exportaciones, nlags=140,method='ywm')
T = len(exportaciones)

sig_test = lambda tau_h: np.abs(tau_h) > 2.58/np.sqrt(T)

In [ ]:
pacf

In [ ]:
for i in range(len(pacf)):
    if sig_test(pacf[i]) == False:
        n_steps = i - 1
        print('n_steps set to', n_steps)
        break

Observamos que con el **pacf** se nos recomienda usar dos retardos, lo cuál nos recueda que cuando se quizo establecer una componente estacional se sugeria un periodo de 2.4, por lo tanto teniendo encuenta lo anterior usaremos **2 retrasos** para la serie original

# Árboles de decisión

### Creación de los rezagos

Debido al análisis previo tomaremos los rezagos de 2 días atrás para poder predecir un paso adelante.


In [ ]:
from pandas import DataFrame
# reframe as supervised learning
# lag observation (t-1) is the input variable and t is the output variable.
df1 = DataFrame() # original
print(df1)
df2 = DataFrame() #diferenciada
print(df2)

In [ ]:
# arreglo de datos para el arreglo de rezagos Serie Original
indice = pd.date_range(start='1/1/2000', periods=282, freq='M')
print(indice)
originalDatadf = pd.DataFrame(data['Total'].values,index=indice)
print(originalDatadf)

In [ ]:
# Rezagos original
for i in range(2,0,-1):
    df1[['t-'+str(i)]] = originalDatadf.shift(i)
print(df1)

In [ ]:
# Create column t original
df1['t'] = originalDatadf.values
print(df1.head(14))

In [ ]:
# Create a new subsetted dataframe, removing Nans from first 3 rows original
df1_Ori = df1[2:]
print(df1_Ori)
df1_Ori.size

In [ ]:
# Split data Serie Original
Orig_Split = df1_Ori.values
# split into lagged variables and original time series
X1 = Orig_Split[:, 0:-1]  # slice all rows and start with column 0 and go up to but not including the last column
y1 = Orig_Split[:,-1]  # slice all rows and last column, essentially separating out 't' column
print(X1)
print('Respuestas \n',y1)

# Árbol para Serie Original

#### Entrenamiento, Validación y prueba


In [ ]:
Y1 = y1
print('Complete Observations for Target after Supervised configuration: %d' %len(Y1))
traintarget_size = int(len(Y1) * 0.70) 
valtarget_size = int(len(Y1) * 0.10)+1# Set split
testtarget_size = int(len(Y1) * 0.20)# Set split
print(traintarget_size,valtarget_size,testtarget_size)
print('Train + Validation + Test: %d' %(traintarget_size+valtarget_size+testtarget_size))

In [ ]:
# Target Train-Validation-Test split(70-10-20)
train_target, val_target,test_target = Y1[0:traintarget_size], Y1[(traintarget_size):(traintarget_size+valtarget_size)],Y1[(traintarget_size+valtarget_size):len(Y1)]

print('Observations for Target: %d' % (len(Y1)))
print('Training Observations for Target: %d' % (len(train_target)))
print('Validation Observations for Target: %d' % (len(val_target)))
print('Test Observations for Target: %d' % (len(test_target)))

In [ ]:
# Features Train--Val-Test split

trainfeature_size = int(len(X1) * 0.70)
valfeature_size = int(len(X1) * 0.10)+1# Set split
testfeature_size = int(len(X1) * 0.20)# Set split
train_feature, val_feature,test_feature = X1[0:traintarget_size],X1[(traintarget_size):(traintarget_size+valtarget_size)] ,X1[(traintarget_size+valtarget_size):len(Y1)]

print('Observations for Feature: %d' % (len(X1)))
print('Training Observations for Feature: %d' % (len(train_feature)))
print('Validation Observations for Feature: %d' % (len(val_feature)))
print('Test Observations for Feature: %d' % (len(test_feature)))

### Árbol


In [ ]:
# Decision Tree Regresion Model
from sklearn.tree import DecisionTreeRegressor
# Create a decision tree regression model with default arguments
decision_tree_Orig = DecisionTreeRegressor()  # max-depth not set
# The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.
# Fit the model to the training features(covariables) and targets(respuestas)
decision_tree_Orig.fit(train_feature, train_target)
# Check the score on train and test
print("Coeficiente R2 sobre el conjunto de entrenamiento:",decision_tree_Orig.score(train_feature, train_target))
print("Coeficiente R2 sobre el conjunto de Validación:",decision_tree_Orig.score(val_feature,val_target))  # predictions are horrible if negative value, no relationship if 0
print("el RECM sobre validación es:",(((decision_tree_Orig.predict(val_feature)-val_target)**2).mean()) )

Vemos que el R2 para los datos de validación es bueno así sin ningún ajuste, Se relizara un ajuste de la profundidad como hiperparametro para ver si mejora dicho valor


In [ ]:
# Find the best Max Depth

# Loop through a few different max depths and check the performance
# Try different max depths. We want to optimize our ML models to make the best predictions possible.
# For regular decision trees, max_depth, which is a hyperparameter, limits the number of splits in a tree.
# You can find the best value of max_depth based on the R-squared score of the model on the test set.

for d in [2, 3, 4, 5,6,7,8,9,10,11,12,13,14,15]:
    # Create the tree and fit it
    decision_tree_Orig = DecisionTreeRegressor(max_depth=d)
    decision_tree_Orig.fit(train_feature, train_target)
    
    # Print out the scores on train and test
    print('max_depth=', str(d))
    print("Coeficiente R2 sobre el conjunto de entrenamiento:",decision_tree_Orig.score(train_feature, train_target))
    print("Coeficiente R2 sobre el conjunto de validación:",decision_tree_Orig.score(val_feature, val_target), '\n')  # You want the test score to be positive and high
    print("el RECM sobre el conjunto de validación es:",sklearn.metrics.mean_squared_error(decision_tree_Orig.predict(val_feature),val_target, squared=False))

Note que los scores para el conjunto de validación son negativos para todas las profundidades evaluadas. Ahora uniremos validacion y entrenamiento para re para reestimar los parametros


In [ ]:
print(type(train_feature))
print(type(val_feature))
#######
print(type(train_target))
print(type(val_target))
####
print(train_feature.shape)
print(val_feature.shape)
#####
####
print(train_target.shape)
print(val_target.shape)
###Concatenate Validation and test
train_val_feature=np.concatenate((train_feature,val_feature),axis=0)
train_val_target=np.concatenate((train_target,val_target),axis=0)
print(train_val_feature.shape)
print(train_val_target.shape)

In [ ]:
# Use the best max_depth 
decision_tree_Orig = DecisionTreeRegressor(max_depth=4)  # fill in best max depth here
decision_tree_Orig.fit(train_val_feature, train_val_target)

# Predict values for train and test
train_val_prediction = decision_tree_Orig.predict(train_val_feature)
test_prediction = decision_tree_Orig.predict(test_feature)

# Scatter the predictions vs actual values
plt.scatter(train_val_prediction, train_val_target, label='train')  # blue
plt.scatter(test_prediction, test_target, label='test')  # orange
# Agrega títulos a los ejes
plt.xlabel('Valores Predichos')  # Título para el eje x
plt.ylabel('Valores Objetivo')  # Título para el eje y
# Muestra una leyenda
plt.legend()
plt.show()
print("Raíz de la Pérdida cuadrática Entrenamiento:",sklearn.metrics.mean_squared_error( train_val_prediction, train_val_target,squared=False))

print("Raíz de la Pérdida cuadrática Prueba:",sklearn.metrics.mean_squared_error(test_prediction, test_target,squared=False))

In [ ]:
from sklearn import tree

listacaract=list(df1_Ori.columns.values)
respuesta=listacaract.pop()
text_representation = tree.export_text(decision_tree_Orig)
print(text_representation)

In [ ]:
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(decision_tree_Orig, 
                   feature_names=listacaract,  
                   class_names=[respuesta],
                   filled=True)

Ahora miraremos las predicciones comparadas con los valores verdaderos, para ver más claro lo anterior.


In [ ]:
print(train_val_prediction.size)
print(train_val_target.size)

print(test_prediction.size)
print(test_target.size)

In [ ]:
indicetrian_val_test=df1_Ori.index
print(indicetrian_val_test.size)  ###Tamaño del índice
indicetrain_val=indicetrian_val_test[0:225]
indicetest=indicetrian_val_test[225:280]

In [ ]:
print(indicetrain_val.size)
print(indicetest.size)

In [ ]:
targetjoint=np.concatenate((train_val_target,test_target))
predictionjoint=np.concatenate((train_val_prediction,test_prediction))
print(targetjoint.size)
print(predictionjoint.size)

In [ ]:
d = {'observado': targetjoint, 'Predicción': predictionjoint}
ObsvsPred1=pd.DataFrame(data=d,index=indicetrian_val_test)
ObsvsPred1.head(10)

In [ ]:
#gráfico
ax = ObsvsPred1['observado'].plot(marker="o", figsize=(10, 6), linewidth=1, markersize=4)  # Ajusta el grosor de las líneas y puntos
ObsvsPred1['Predicción'].plot(marker="o", linewidth=1, markersize=2, ax=ax)  # Ajusta el grosor de las líneas y puntos
# Agrega una línea vertical roja
ax.axvline(x=indicetrian_val_test[223].date(), color='red', linewidth=0.5)  # Ajusta el grosor de la línea vertical
# Muestra una leyenda
plt.legend()
plt.show()

In [ ]:
#metrica
print("Raíz de la Pérdida cuadrática Entrenamiento:",sklearn.metrics.mean_squared_error( ObsvsPred1['Predicción'][:223], ObsvsPred1['observado'][:223],squared=False))
print("Raíz de la Pérdida cuadrática Prueba:",sklearn.metrics.mean_squared_error(ObsvsPred1['Predicción'][223:], ObsvsPred1['observado'][223:],squared=False))

# Serie de Exportaciones sin Tendencia

Implementaremos ahora el modelo de árboles sobre la serie sin tendencia, eliminada usando la estimación dada por medio del filtro de promedios móviles. Vamos a importar la bases de datos y a convertirlas en objetos de series de Tiempo. $\{X_t\}$


In [ ]:
# Lectura de la serie
data2 = pd.ExcelFile('ExportacionesSinTendencia.xlsx')
print(data2.sheet_names)
# dataframe con los datos
data2 = data2.parse('Sheet1')
print(data2)
print(type(data2))

In [ ]:
# tipo de datos
print(data2.info())
# nombre
data2.columns = ['Fecha','Valor']
print(data2.info())

In [ ]:
# fecha
data2['Fecha']=pd.to_datetime(data2['Fecha'])###Sólo es necesario si no tiene formato de fecha
dlData = data2.set_index('Fecha')
#dlData=dlData.asfreq('M')
print(type(dlData))
print(dlData)

In [ ]:
#mirando los datos
#objeto ts
exportaciones = dlData['Valor']
print(type(exportaciones))
plt.plot(exportaciones)

In [ ]:
print(f'Numero de filas con valores faltantes: {data2.isnull().any(axis=1).mean()}')

In [ ]:
dlData.shape

## PACF

usaremos la funcion de autocorrealcion parcial para darnos una idea de cuantos rezagos usaremos en el modelo


In [ ]:
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(exportaciones,lags=134,method='ywm',alpha=0.01)
pyplot.show()

In [ ]:
import statsmodels.api as sm
pacf =  sm.tsa.stattools.pacf(exportaciones, nlags=134,method='ywm')
T = len(exportaciones)

sig_test = lambda tau_h: np.abs(tau_h) > 2.58/np.sqrt(T)

In [ ]:
pacf

In [ ]:
for i in range(len(pacf)):
    if sig_test(pacf[i]) == False:
        n_steps = i - 1
        print('n_steps set to', n_steps)
        break

Observamos que con el **pacf** se nos recomienda usar un solo retardo, usaremos **1 retraso** para la serie sin Tendencia

# Árboles de decisión

### Creación de los rezagos

Debido al análisis previo tomaremos los rezagos de 1 días atrás para poder predecir un paso adelante.


In [ ]:
from pandas import DataFrame
# reframe as supervised learning
# lag observation (t-1) is the input variable and t is the output variable.
df1 = DataFrame() # original
print(df1)
df2 = DataFrame() #diferenciada
print(df2)

In [ ]:
# arreglo de datos para el arreglo de rezagos Serie Original
indice = pd.date_range(start='2/1/2000', periods=281, freq='MS')
print(indice)
difDatadf = pd.DataFrame(data2['Valor'].values,index=indice)
print(difDatadf)

In [ ]:
# Rezagos original
for i in range(1,0,-1):
    df2[['t-'+str(i)]] = difDatadf.shift(i)
print(df2)

In [ ]:
# Create column t original
df2['t'] = difDatadf.values
print(df2.head(14))

In [ ]:
# Create a new subsetted dataframe, removing Nans from first 3 rows original
df2_dif = df2[1:]
print(df2_dif)
df2_dif.size

In [ ]:
# Split data Serie Original
dif_Split = df2_dif.values
# split into lagged variables and original time series
X2 = dif_Split[:, 0:-1]  # slice all rows and start with column 0 and go up to but not including the last column
y2 = dif_Split[:,-1]  # slice all rows and last column, essentially separating out 't' column
print(X2)
print('Respuestas \n',y2)

# Árbol para Serie Sin Tendencia

#### Entrenamiento, Validación y prueba


In [ ]:
Y1 = y1
print('Complete Observations for Target after Supervised configuration: %d' %len(Y1))
traintarget_size = int(len(Y1) * 0.70) 
valtarget_size = int(len(Y1) * 0.10)+1# Set split
testtarget_size = int(len(Y1) * 0.20)# Set split
print(traintarget_size,valtarget_size,testtarget_size)
print('Train + Validation + Test: %d' %(traintarget_size+valtarget_size+testtarget_size))

In [ ]:
# Target Train-Validation-Test split(70-10-20)
train_target, val_target,test_target = Y1[0:traintarget_size], Y1[(traintarget_size):(traintarget_size+valtarget_size)],Y1[(traintarget_size+valtarget_size):len(Y1)]

print('Observations for Target: %d' % (len(Y1)))
print('Training Observations for Target: %d' % (len(train_target)))
print('Validation Observations for Target: %d' % (len(val_target)))
print('Test Observations for Target: %d' % (len(test_target)))

In [ ]:
# Features Train--Val-Test split

trainfeature_size = int(len(X1) * 0.70)
valfeature_size = int(len(X1) * 0.10)+1# Set split
testfeature_size = int(len(X1) * 0.20)# Set split
train_feature, val_feature,test_feature = X1[0:traintarget_size],X1[(traintarget_size):(traintarget_size+valtarget_size)] ,X1[(traintarget_size+valtarget_size):len(Y1)]

print('Observations for Feature: %d' % (len(X1)))
print('Training Observations for Feature: %d' % (len(train_feature)))
print('Validation Observations for Feature: %d' % (len(val_feature)))
print('Test Observations for Feature: %d' % (len(test_feature)))

### Árbol


In [ ]:
# Decision Tree Regresion Model
from sklearn.tree import DecisionTreeRegressor
# Create a decision tree regression model with default arguments
decision_tree_Orig = DecisionTreeRegressor()  # max-depth not set
# The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.
# Fit the model to the training features(covariables) and targets(respuestas)
decision_tree_Orig.fit(train_feature, train_target)
# Check the score on train and test
print("Coeficiente R2 sobre el conjunto de entrenamiento:",decision_tree_Orig.score(train_feature, train_target))
print("Coeficiente R2 sobre el conjunto de Validación:",decision_tree_Orig.score(val_feature,val_target))  # predictions are horrible if negative value, no relationship if 0
print("el RECM sobre validación es:",(((decision_tree_Orig.predict(val_feature)-val_target)**2).mean()) )

Vemos que el R2 para los datos de validación es malo pues  es negativo, Se relizará un ajuste de la profundidad como hiperparametro para ver si mejora dicho valor


In [ ]:
# Find the best Max Depth

# Loop through a few different max depths and check the performance
# Try different max depths. We want to optimize our ML models to make the best predictions possible.
# For regular decision trees, max_depth, which is a hyperparameter, limits the number of splits in a tree.
# You can find the best value of max_depth based on the R-squared score of the model on the test set.

for d in [2, 3, 4, 5,6,7,8,9,10,11,12,13,14,15]:
    # Create the tree and fit it
    decision_tree_Orig = DecisionTreeRegressor(max_depth=d)
    decision_tree_Orig.fit(train_feature, train_target)
    
    # Print out the scores on train and test
    print('max_depth=', str(d))
    print("Coeficiente R2 sobre el conjunto de entrenamiento:",decision_tree_Orig.score(train_feature, train_target))
    print("Coeficiente R2 sobre el conjunto de validación:",decision_tree_Orig.score(val_feature, val_target), '\n')  # You want the test score to be positive and high
    print("el RECM sobre el conjunto de validación es:",sklearn.metrics.mean_squared_error(decision_tree_Orig.predict(val_feature),val_target, squared=False), '\n')

Note que los scores para el conjunto de validación son negativos para todas las profundidades evaluadas. Tomaremos el más cercano a cero que el el de la profundidad 2. Ahora uniremos validacion y entrenamiento para re para reestimar los parametros


In [ ]:
print(type(train_feature))
print(type(val_feature))
#######
print(type(train_target))
print(type(val_target))
####
print(train_feature.shape)
print(val_feature.shape)
#####
####
print(train_target.shape)
print(val_target.shape)
###Concatenate Validation and test
train_val_feature=np.concatenate((train_feature,val_feature),axis=0)
train_val_target=np.concatenate((train_target,val_target),axis=0)
print(train_val_feature.shape)
print(train_val_target.shape)

In [ ]:
# Use the best max_depth 
decision_tree_Orig = DecisionTreeRegressor(max_depth=2)  # fill in best max depth here
decision_tree_Orig.fit(train_val_feature, train_val_target)

# Predict values for train and test
train_val_prediction = decision_tree_Orig.predict(train_val_feature)
test_prediction = decision_tree_Orig.predict(test_feature)

# Scatter the predictions vs actual values
plt.scatter(train_val_prediction, train_val_target, label='train')  # blue
plt.scatter(test_prediction, test_target, label='test')  # orange
# Agrega títulos a los ejes
plt.xlabel('Valores Predichos')  # Título para el eje x
plt.ylabel('Valores Objetivo')  # Título para el eje y
# Muestra una leyenda
plt.legend()
plt.show()
print("Raíz de la Pérdida cuadrática Entrenamiento:",sklearn.metrics.mean_squared_error( train_val_prediction, train_val_target,squared=False))

print("Raíz de la Pérdida cuadrática Prueba:",sklearn.metrics.mean_squared_error(test_prediction, test_target,squared=False))

In [ ]:
from sklearn import tree

listacaract=list(df1_Ori.columns.values)
respuesta=listacaract.pop()
text_representation = tree.export_text(decision_tree_Orig)
print(text_representation)

In [ ]:
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(decision_tree_Orig, 
                   feature_names=listacaract,  
                   class_names=[respuesta],
                   filled=True)

Ahora miraremos las predicciones comparadas con los valores verdaderos, para ver más claro lo anterior.


In [ ]:
print(train_val_prediction.size)
print(train_val_target.size)

print(test_prediction.size)
print(test_target.size)

In [ ]:
indicetrian_val_test=df1_Ori.index
print(indicetrian_val_test.size)  ###Tamaño del índice
indicetrain_val=indicetrian_val_test[0:215]
indicetest=indicetrian_val_test[215:269]

In [ ]:
print(indicetrain_val.size)
print(indicetest.size)

In [ ]:
targetjoint=np.concatenate((train_val_target,test_target))
predictionjoint=np.concatenate((train_val_prediction,test_prediction))
print(targetjoint.size)
print(predictionjoint.size)

In [ ]:
d = {'observado': targetjoint, 'Predicción': predictionjoint}
ObsvsPred2=pd.DataFrame(data=d,index=indicetrian_val_test)
ObsvsPred2.tail(54)

In [ ]:
#gráfico
ax = ObsvsPred2['observado'].plot(marker="o", figsize=(10, 6), linewidth=1, markersize=4)  # Ajusta el grosor de las líneas y puntos
ObsvsPred2['Predicción'].plot(marker="o", linewidth=1, markersize=2, ax=ax)  # Ajusta el grosor de las líneas y puntos
# Agrega una línea vertical roja
ax.axvline(x=indicetrian_val_test[223].date(), color='red', linewidth=0.5)  # Ajusta el grosor de la línea vertical
# Muestra una leyenda
plt.legend()
plt.show()

Ahora nos devolveremos a la escala original


In [ ]:
#formando la de vuelta a la escala de los datos 
# datos transfromados para la diferencia
# traer datos
data_3 = pd.ExcelFile('datos/LExportaciones.xlsx')
print(data_3.sheet_names)
# dataframe con los datos
data_3 = data_3.parse('Sheet 1')
print(data_3)

In [ ]:
#mirando los datos
#objeto ts
# nombre
data_3.columns = ['Fecha','Valor']
# fecha
data_3['Fecha']=pd.to_datetime(data_3['Fecha'])###Sólo es necesario si no tiene formato de fecha
LExportaciones = data_3.set_index('Fecha')
#LExportaciones=LExportaciones.asfreq('M')
print(type(LExportaciones))
LExportaciones = LExportaciones.drop(['2017-01-01'])
# para tipo de dato
indiceL = LExportaciones.index
d = {'observado L': LExportaciones['Valor']}
ObsL=pd.DataFrame(data=d,index=indiceL)
ObsL.head(10)

Ahora devolvemos la diferenciación

In [ ]:
#necesitaremos arreglar los indices
indices = ObsL['observado L'][6:275].index
ObsvsPred2['Predicción'].index = indices
# aqui devolvemos la diferenciación en escala log
yhat_Dif = ObsvsPred2['Predicción'] + ObsL['observado L'][6:275]
#yhat_Dif = yhat_Dif.drop('2000-07-01')
print(yhat_Dif)

In [ ]:
#volver a escala normal
yhat = (((0.45)*yhat_Dif) + 1)**(1/0.45)
print(yhat)

In [ ]:
y = ObsvsPred1['observado'][6:275]
#necesitaremos arreglar los indices
indices = ObsL['observado L'][6:275].index
y.index = indices
print(y)
#y = y.drop('2017-01-03')
d = {'observado': y, 'Predicción': yhat}
ObsvsPred3=pd.DataFrame(data=d,index=indices)
ObsvsPred3.head(10)

In [ ]:
#gráfico
ax = ObsvsPred3['observado'].plot(marker="o", figsize=(10, 6), linewidth=2, markersize=4)  # Ajusta el grosor de las líneas y puntos
ObsvsPred3['Predicción'].plot(marker="o", linewidth=1, markersize=2, ax=ax)  # Ajusta el grosor de las líneas y puntos
# Agrega una línea vertical roja
ax.axvline(x=indicetrian_val_test[223].date(), color='red', linewidth=0.5)  # Ajusta el grosor de la línea vertical
# Muestra una leyenda
plt.legend()
plt.show()

In [ ]:
#metrica
print("Raíz de la Pérdida cuadrática Entrenamiento:",sklearn.metrics.mean_squared_error( yhat[:222], y[:222],squared=False))
print("Raíz de la Pérdida cuadrática Prueba:",sklearn.metrics.mean_squared_error(yhat[222:], y[222:],squared=False))